In [1]:
%%configure -f
{
 "executorCores": 4, 
 "executorMemory": "47696M",
 "conf": {"spark.default.parallelism": 1000,
          "spark.sql.shuffle.partitions": 1000,
          "spark.task.cpus": 1
         }
}

## TODO:
* extend company_description matrix to include all company info
* check if this does a better job with recall just with SVD then revenue-ebitda reranking

In [1]:
model_path = "s3://onai-ml-dev-eu-west-1/company2vec/model_description_only/svd"
word2id_path = "s3://onai-ml-dev-eu-west-1/company2vec/common"

In [2]:
from gensim.utils import lemmatize, tokenize
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

from langdetect import detect
import string
from itertools import islice
import numpy as np
import smart_open
from collections import Counter, defaultdict
import math
import json

from pyspark.ml.feature import (HashingTF,
                                IDF,
                                Tokenizer,
                                StopWordsRemover,
                                CountVectorizer,
                                StringIndexer,
                                OneHotEncoderEstimator,
                                VectorAssembler,
                                VectorSizeHint,
                                StandardScaler,
                                PCA
                               )
from pyspark.ml import Pipeline, Transformer

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T

from pyspark.mllib.linalg import DenseMatrix
from pyspark.mllib.linalg.distributed import RowMatrix
from pyspark.mllib.linalg import Vector as MLLibVector, Vectors as MLLibVectors

from pyspark.ml.linalg import Vectors, VectorUDT

import smart_open
import faiss
import requests

Loading faiss with AVX2 support.


In [3]:
import graphene

In [14]:
import os
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search, Index
import numpy as np
import ssl
from elasticsearch.connection import create_ssl_context
from elasticsearch_dsl import Q

import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

public_labels = {
    'da166d49-acbc-54c5-9ab4-d72a143920f6': ['7006ce31-6d5d-5217-b993-8865e3cea407', 'a2e7f4cf-6372-5ba8-a4c5-d05c7fda8b1e', '474515ad-010b-52be-941d-96cecb721140', 'c17d9a27-1622-50d9-83f2-e7846575f3f6'],
    'e007e0c6-91b3-55f9-b254-283017edf637': ['dcaaffbe-2b69-5cda-8291-9b7eeb014ed3', '4fa675cd-2313-5b10-8743-183e201ea0e4', 'dc1b52f1-0e78-5715-b472-d2763d6a885d'],
    '42f9261d-f978-54f5-8e4b-9f22021311e1': ['d1f5b446-b90e-56be-8ad6-31ba3fadd3ba', 'bc01fd71-dfd4-5c17-ac67-bb9359e14197', '7d999d67-4167-5a41-b59a-db1225b50304'],
    '9455d310-201a-5545-b073-fecbc49cbd94': ['ff0d0ab5-2e35-5f31-afba-424acb977718', 'b70f3942-69cd-5332-9ef2-2aa558a2f57d'],
    'dcaaffbe-2b69-5cda-8291-9b7eeb014ed3': ['b70f3942-69cd-5332-9ef2-2aa558a2f57d', 'ce26af83-1c3c-5c53-8fe8-7507f153763a', 'd28ca355-aa12-524d-b878-b41ed1b89819'],
    'd31fa65a-2a47-5162-8724-2abd53abdd74': ['27e8c3a3-8a58-56c5-a88b-89a70b1a87d9', 'f52ec136-691b-5ba5-b1be-a5946cdf128c', '74c0fe70-2803-5e61-b172-fa3dacb1fd9b', 'e007e0c6-91b3-55f9-b254-283017edf637'],
    'b8418db0-b87c-5b38-8dcd-dd22efab15bc': ['d8b42036-1486-50e1-ac57-93a62c234a3d', 'eae44ab1-7f39-5eba-94ad-741766056496', 'be563bca-c715-5ce0-bafb-801a126f61eb', 'dfb4a9c3-37fd-538c-b4a2-1b3ea43e4879', '4e90aaa0-552e-5dc1-b1c9-1eab2104da86', '14863124-ae2b-56bd-b5a9-3269e2f686ab', '20a1edcf-81dc-5736-a863-46d9ba3e79e6', 'a8183ab9-60ef-5bbe-9150-570ce9579a9c', '50930989-84f0-5b0b-9bb8-c5bc52982a9f', '3e761817-0a9d-52bc-af19-58ffc7fb8623'],
    'd754b64d-5e88-5134-9b44-fa9839413312': ['bc32a40b-1dec-538d-bff8-7a445d6c3690', '1fc6d921-ad3e-5588-997a-70d015e0a57c', '58ea2029-83f4-5ce9-a330-2d40c816549d'],
    'e631bb16-4779-5176-8ad4-7d95375910a0': ['f6910844-d5e8-53f6-9352-121a9aef5344', '9b85c07f-0291-5f40-878c-92920d612aca', '75aaf1f7-ebc1-5c58-ba12-6194e560c4fc', 'c0efe3cf-6668-51c2-9443-f10b2994118a', 'f6fa6a09-7ce1-5c24-8043-569eaf71a51e', '8ab6a9d3-674f-599e-9986-7c176ab56c78', '95b851bc-621e-5351-96f8-75d4e4828534'],
    '532011cb-5ba0-595a-98b6-72b9cf6bf6c5': ['ace38e9d-84f8-511c-8557-fbf309605037', '29f36706-286f-5e1f-8fed-95e257424ff6', '4734141d-ad19-53c6-a872-eb167fe41a97', '4a98176d-c1d2-56cd-a67b-94d890e4ec95', '461baa1c-012f-55cd-b052-6fc707c84928'],
    'fe7f9eee-19f9-5d22-a56f-d94c8aa1059b': ['f3de1e65-a9ea-5115-b81e-24d488c68817', '37b40752-4f2d-5a64-a5de-ebdbfc49eb41'],
    '1eb5a029-297f-564d-a25f-0553cc689aa0': ['dbfae6fd-61bd-5857-8d56-f3418b19dd16', 'aaf82bc0-993f-56a9-a1da-db2a6bc19914', '0374e5c2-3db8-5fa6-84d4-15aee8857a8e', '15067bfc-3d78-50ec-9afe-8b0efc6bacb3', '9653f7ab-cb79-5073-a875-53946ae55876', 'd72ee998-9877-525b-9a96-173d93845cb2'],
    'b085b1f3-be9e-58a9-8608-91ebe9add601': ['ede76d6d-5eb6-517a-8c3a-26f81d11b07f', '40a3b357-f67c-5612-8159-2c2b05a4e40f', 'f2612ee3-8c72-5ca8-8ec4-3cd69b5c1d29', '08da1506-be21-51d5-a0a8-618622e66972', '92163b47-76a1-51b2-ad9b-1f276a5e8bc7'],
    '874a81b2-a22b-57a6-bf47-813d2fe099c8': ['8945d7fe-39be-5acb-8828-449cbfc2211f', '754562cb-c4b7-5636-9559-4b5e378e879d', '0b8ae7a2-9838-5d5e-a72b-bde8fdcb88eb', 'a5593f19-a714-56d8-85d5-6cc0d6bcbbae', 'ee5bfaab-27e5-53e8-bddc-b0100a796397', 'a4b8e58e-a0c1-51f9-9c9c-b514175b0719'],
    '69bcb95a-0fd0-56d6-be4c-c8614dc18a52': ['254f4ab7-19c2-54e1-8596-bac033ce12cc', 'c205c36f-065e-56f6-8f50-6b64970c75c8', 'ffa88606-4a20-5919-a9bd-96499d647a9a', 'dfe53332-be4a-531f-a4fa-f0451398be97', 'aeb637db-8a46-5803-bf31-974dfe37af53', 'fa107880-e401-5942-92e8-9c12d9bd8496', '0f05d245-c156-5b60-bbcf-478a4b471ca1'],
    'd4b2920d-5b87-525e-915f-d30ad4282a77': ['4eccd10f-3a2d-57be-9b39-de29dfc8e439', 'c83e727c-419e-58fd-8e70-5a8b83a26aa4', 'c819ecfc-0798-55e7-8069-6beb1c8510f2', '7cb28b86-cebd-53e8-8830-0ad96d2dcdee', 'bc6ca52b-ce4c-5593-883c-15ceb2ecd679'],
    'be471a0b-3336-561e-8bcd-05bd54472676': ['35aaf11e-c584-5062-b715-ce65d549baa9', '379a2f48-3ec2-5a9f-9559-84793e824bc6', 'c96d6165-bc7b-5bb3-98cf-3511964ff4a0', 'b4f8e550-9bb3-5358-bd60-67a723188274', '4d0cdcd1-c38d-5dc3-adb1-5d13180b64d6', '302bc7c6-cbd3-5c38-bf05-88b43da69dec', 'c2c1494a-1d28-5af5-acb7-fcd98f3cd63c', '0104e241-2603-5a23-9c4f-22328fe8fa3a'],
    'a4b8e58e-a0c1-51f9-9c9c-b514175b0719': ['05b187c3-af2c-567e-9c9c-e82861872e6d', 'f6d7df84-919e-5cfa-be8a-52a1d0867db7', 'e72381d1-fd10-53ae-a00e-cc25c919bf52', 'a1f01327-ef34-548a-af0c-2bb9b78e9bd6', '67fe9a89-8e60-5b87-9803-5ba17eadfdc0', 'aed7cc97-d89e-58f8-8a90-37d01095fca5', 'a11c947f-a1b0-5584-bf6c-14d49a22f05a', '8a834da6-3895-5aac-821f-b2c82b610936', 'a2774e16-55c3-5a5d-8a98-c21cd16c38c7'],
    '1cbb796a-c57a-575f-8503-7401814d80f1': ['db5c093d-8575-555c-9eea-98729673d56d', 'b7701c74-0edb-5046-b3e0-cda5b24a4f5a', '847d16f8-03a7-5597-8639-c5f3637984ec', '48051655-9ddb-5b87-8ddb-271ccb81fd4e', '51fab99c-a8bf-5020-a03d-3fad50708d12', '823fbf5a-5418-5a7f-9969-169e6adce495', 'db45d8cf-5ee7-5fe9-ad05-5c6218cb1d24', 'ddf225c5-a4fe-5987-b418-c295d8bbf969', 'ed07169f-dfaa-5ef7-8882-af8da6990793', '13f10a0b-290d-50de-9dd4-022bcc1005cd'],
    '37b40752-4f2d-5a64-a5de-ebdbfc49eb41': ['f3de1e65-a9ea-5115-b81e-24d488c68817', 'fe7f9eee-19f9-5d22-a56f-d94c8aa1059b', '811fcd9a-5420-5c85-aaa5-33d9e1f7262d', 'dcaaffbe-2b69-5cda-8291-9b7eeb014ed3', '0cbc1fc7-a10d-5331-8f0b-c43a89af189a', '55a444f5-35e4-5ff1-8e4b-7f565ed2ff5a', 'd003cfc2-6d08-5190-8fb5-7b7f12510d15'],
    '74a0bc09-c2db-56d2-a985-6489ee0f2013': ['e2818957-53aa-5ea7-8be5-df0e6b801f89', '21a0c8bd-8756-5d57-b50d-fb3a09f08648', 'd2c38d8e-241f-5756-85fb-93d483fa5ef9', '1d49d6d3-e73e-57d0-b839-12260300b283', 'fa2bc9fa-25e6-5f0d-b65a-8137b522329e'],
    'e5c74fc1-d13c-5538-a356-a157e67fbd95': ['37e65fdb-7e04-5c64-95e4-50de546533fa', 'c3b5ca3b-136e-56c5-8aff-21bdc1b462af', 'f7eeb53a-7f54-5a0a-afa4-0cd47ed91f93', 'd37aeab7-b52f-5324-adad-ac469d89098e', 'dc0a6209-9bec-58dc-9994-a145d01ecdf5'],
    '0a4ac21f-bee8-59e9-8939-a0a4d820839f': ['a4f143e9-29f3-5de6-a944-0081bf31b9e2', 'b0b3dc1d-3324-5649-b0c9-84ddb2825eca', 'd546098f-941a-5235-b444-9908538682d5', 'fbe69d57-682f-559f-a65c-e077e5c19e93', 'f426a8e0-3916-5adf-894f-66c9ce76a403'],
    '80a3e043-d029-53a7-9877-b693b529960c': ['929441f1-50e5-50b7-a44a-1e215175f17f', '6fb74030-3209-5240-a3e3-e838c0076977', '0c4387af-c1d3-5144-a994-a32e6ce5a160', '847390e5-6df9-5dd8-ba62-220dcd90527d', 'cce2c2a0-1676-5351-8c12-94659ed5da20', '0c376306-835b-5a72-a6ef-1d1b25450aeb', '678372f3-281f-5680-aabc-b4484edd5258', '03331f30-6393-57f0-a39e-5a4599fd8ad4', '94963d48-fcaa-522d-9d26-bd10f4b93e96'],
    '32f946ef-e846-536c-aa9b-ff2e9154b030': ['40dff0db-b22c-5110-96bb-b17c04cf018a', '584c0f9e-a515-5104-820b-29a7067c2977', '2f6df721-f9b2-5f16-b717-8b18d739df87', 'c3a51afe-9783-57af-bb5a-8e9d74d83786'],
    '519f6dd1-96d7-5c57-ad53-f6cb02a6088f': ['a0e34f50-e481-5f5b-b510-090a9a413c03', 'a20cfeae-c58d-5a7a-b603-fc5787e97acc', '59b6061f-8cfc-53d1-94c3-58a756c8ec5c', '5498eb8f-9968-55c1-ace9-d6c1d46a6acf', '61b83321-01b9-5ba2-bd94-4d9785a73657', '20b10f6a-0e61-5ef5-82b4-726f2a2e6631'],
    '025d3ae2-dbcc-56f3-8be7-cd6ae98c3ea4': ['0a910cf9-677d-5ac6-9cce-903cbb14573a', '9e9e54f3-4236-5856-9850-d3387b148d66', '748446f0-49a0-5597-a5c9-9988576619d3', 'af5ddf33-af4f-5454-a1cf-fcd68af59d38', '1bb25f48-a4e8-504b-8fa7-c7cfc17753d3'],
    'b3dd3855-58c8-53d0-8e39-107b911575bb': ['71cdfe54-7dce-5f1d-9cfd-8e16e56c945b', '02c4ffde-3cb3-5407-a6ed-b60be91328d1', 'e8645a5f-481c-50ed-ab83-213db350a9cb', '67164833-d04b-52fc-85c2-ab1447ca5d93', '3cf98156-62e6-57fc-94c7-e50ed6f24dd0'],
    '95b851bc-621e-5351-96f8-75d4e4828534': ['eb175ee3-f440-5459-96e3-72e800efc11c', '58728f97-f3c0-58fa-a001-69044f0b4fa7', '523abdd8-5fea-555f-afb9-66b85ca25eca', 'e631bb16-4779-5176-8ad4-7d95375910a0', '9b85c07f-0291-5f40-878c-92920d612aca', '75aaf1f7-ebc1-5c58-ba12-6194e560c4fc', 'f6fa6a09-7ce1-5c24-8043-569eaf71a51e', 'f6910844-d5e8-53f6-9352-121a9aef5344', '4ba722f5-e2d8-5517-8e36-9a86707a9c54', '8ab6a9d3-674f-599e-9986-7c176ab56c78'],
    '30e94db2-4762-58bd-83ec-fd866ab80a90': ['97fa0717-519b-5b59-9841-a4c16fabcdc7', 'f50bb1cd-0138-585e-b521-772613f0be33', 'ada60c70-fa99-532b-8280-df2d8b56a2a7', 'deba9dc1-f542-5550-a759-bbd3cebe3518', '89dfb8ce-3556-5777-a788-1a590cd7fae2'],
    '40be829f-4b70-58ee-8b47-41b7b856e867': ['20fe8651-5165-50a1-b245-3abf5714dcff', 'a62daa94-3a87-5cfc-a3dc-d66da1939392', 'a67eab61-226f-5357-a5b3-0ce9446924e6', '61115758-bc76-5f6c-acfa-5aad6e3aac68'],
    'df671f51-4e93-5c03-ad4b-4efcb5e028ed': ['2f9eee2d-0c6c-5ca5-b16d-4c98713548fe', 'c786d929-ac09-5dfd-91b4-9ef44257e3a1', '55ebd5b6-2999-552a-b511-6397ab7fb5c4', '0a33c5e7-5639-5598-a94c-6e78436e59db', '9ca50e4c-c83f-5456-91a8-c681e0937af0', '5a62fb6f-d1c0-53b6-ad94-d3d3cc4179a5', 'b9445d7a-cb1e-5f8e-a155-df163a2937a7'],
    '878ef8f5-88dd-5f88-8df2-4e87e7dc54ce': ['3ba9fa96-c8d5-5c70-b353-de778e028e5a', '09771893-a6bb-5feb-ab51-1c0115fa302b', '6a5c0f1b-f20c-5949-96e4-5bbe3ac23a35', '9e9e54f3-4236-5856-9850-d3387b148d66', '748446f0-49a0-5597-a5c9-9988576619d3', '39e5e158-41e9-53c6-b736-924786f5fb62', '47e614b5-e26d-5801-bdef-967647b7994f', '6526bbe9-b49f-554e-b6ec-cb74509a8447'],
    'ba12210e-8647-5154-ac24-4a3ad82a6490': ['af5ddf33-af4f-5454-a1cf-fcd68af59d38', '4e013d2a-6c5a-52dc-904c-b53f6000c495', '72349f49-0f3f-5b01-b31b-1e39df4e5133', 'f025acfa-becf-519e-86fa-74c433f936bb'],
    'dc1b52f1-0e78-5715-b472-d2763d6a885d': ['e007e0c6-91b3-55f9-b254-283017edf637', '626a6b74-8cbd-5994-91b4-872f0547325c', 'be980fc6-08cb-5547-9ebb-9da52e6e8355', '085b7710-d08a-5869-916f-2ad36d74243c'],
    '19773391-bd97-5e01-a216-367be9c8bb05': ['9e6fd0da-b4fa-590a-8fb8-4955a734b17f', '96c816a4-47ff-585e-8273-ae6ccb71fcde', '5a1cd62f-b86c-5cc6-a5e6-161489cf6a3c', 'edf5992d-2ea6-5d02-b918-17f100989d7f'],
    '302bc7c6-cbd3-5c38-bf05-88b43da69dec': ['0320fc4f-ba0c-55b9-b44a-2685623245cd', '311437ee-0807-5f3e-8ae3-bc78380a0b0a', '379a2f48-3ec2-5a9f-9559-84793e824bc6'],
    'c2c1494a-1d28-5af5-acb7-fcd98f3cd63c': ['9f7e0fea-11f7-5071-bf16-a75121721cba', '72b2be27-3c1f-5027-a248-78fd8d3fd517', 'be471a0b-3336-561e-8bcd-05bd54472676', '4b6bb784-c9e8-59a0-b7a6-67902ceafc7c', 'c96d6165-bc7b-5bb3-98cf-3511964ff4a0'],
    '1fc6d921-ad3e-5588-997a-70d015e0a57c': ['bc32a40b-1dec-538d-bff8-7a445d6c3690', '3155037a-8dec-5e7c-9164-c6d170912652', 'd754b64d-5e88-5134-9b44-fa9839413312', 'a4378116-e0cb-547a-805f-1ede4f751c23', 'bb59024b-00c2-52e4-8cb7-5ff2a0626c86', '3a4a52fc-79fe-5f82-9b7c-bf8f44bd4d48', '2ca38fea-5a18-54f6-bbef-ca30f5aa08fd', '1b7cd4a1-6e5d-5779-9ca5-f3ab6f02b802', 'f36fee5b-f42e-5a00-a509-22784ccecdbb'],
    'b7e11624-05fc-56d8-935b-1f42fa0d4eed': ['1fc6d921-ad3e-5588-997a-70d015e0a57c', 'bc32a40b-1dec-538d-bff8-7a445d6c3690', '3155037a-8dec-5e7c-9164-c6d170912652', 'a4378116-e0cb-547a-805f-1ede4f751c23', '52a1e5d0-05ef-5dca-9f6c-ff915a8cda66', '59b6061f-8cfc-53d1-94c3-58a756c8ec5c', '3a4a52fc-79fe-5f82-9b7c-bf8f44bd4d48', 'ad45c851-53dd-51a7-ab65-2f72f2110698']
}

HOST = "http://host.docker.internal:9200"
INDEX = "company"

In [10]:
word2id = {}
id2word = {}
with smart_open.open(f"{word2id_path}/bow/word2id.csv", "r") as f:
    for line in f:
        word,idd = line.strip().split(",")
        word2id[word] = int(idd)
        id2word[int(idd)] = word

In [11]:
client = Elasticsearch(HOST+"/")

In [80]:
def debug_es(target):
    q = Q('more_like_this',
          like=[{"_id": target, "_index": INDEX}],
          fields=[
              "primary_sic_node_desc.keyword",
              "business_description"
              ],
          min_doc_freq=1,
          min_term_freq=0,
          max_query_terms=15,
          include=True, )

    s = (
        Search(index=INDEX)
        .using(client)
    )
    

    q = Q('bool',
          should=[q]
          )

    s = (s
         .query(q)
         .params(search_type='dfs_query_then_fetch')
         .extra(explain=True)
         ).execute()

    
    def get_keyword(kw_s):
        return kw_s.split(":")[1].split()[0]
    
    hit = s[0:1][0]
    ret = []
    print("#"*40, hit.name, "#"*40)
    for el in hit.meta.explanation.details:
        ret.append((el.value,
                    get_keyword(el.description),
                    el.details[0].details[1].value,
                    el.details[0].details[0].value
                   ))
    print(f"Score: {hit.meta.explanation.details[0].value}")
    for match, term, tf, idf in sorted(ret, reverse=True):
        print(f"{match} {term}, TF:{tf}, IDF:{idf}")
    print("-"*40)
    
    for rank, hit in enumerate(s[1:100]):
        print("-"*40, rank+1, "-"*40)
        
        print(f"ES peer {hit.name}")
        ret = []
        print(f"Score: {hit.meta.explanation.value}")
        for el in hit.meta.explanation.details:
            ret.append((el.value, 
                        get_keyword(el.description),
                        el.details[0].details[1].value,
                        el.details[0].details[0].value
                       ))
        for match, term, tf, idf in sorted(ret, reverse=True):
            print(f"{match} {term}, TF:{tf}, IDF:{idf}")
        print("-"*40)

In [81]:
for target in islice(public_labels, 1):
    debug_es(target)

######################################## AT & S Austria Technologie & Systemtechnik Aktiengesellschaft ########################################
Score: 5.872923
13.344405 pcbs, TF:9.241963, IDF:2.2
9.85669 hdi, TF:9.954675, IDF:2.2
8.356614 systemtechnik, TF:10.7603, IDF:2.2
7.769165 substrates, TF:7.8463974, IDF:2.2
7.278677 technologie, TF:9.372306, IDF:2.2
7.112833 Printed, TF:7.112833, IDF:2.2
6.2014904 flexible, TF:6.2631392, IDF:2.2
6.0087204 circuit, TF:6.068453, IDF:2.2
5.872923 microvia, TF:12.454895, IDF:2.2
5.860599 rigid, TF:7.546334, IDF:2.2
5.2753835 austria, TF:5.327826, IDF:2.2
5.1072793 aktiengesellschaft, TF:6.57633, IDF:2.2
4.8222313 printed, TF:6.2092915, IDF:2.2
4.634316 boards, TF:5.9673243, IDF:2.2
4.451558 copper, TF:5.7319984, IDF:2.2
0.87032723 and, TF:0.55672854, IDF:2.2
----------------------------------------
---------------------------------------- 1 ----------------------------------------
ES peer AT&S Korea Co., Ltd.
Score: 59.72927
7.311725 circuit, TF:6